In [1]:
#Michael Osborne
#CSCI 191T Machine Learning: Titanic Assignment 
import pandas as pd
import numpy as np
import os
from sklearn.tree import DecisionTreeClassifier

print(os.listdir())

['submission.csv', 'gender_submission.csv', 'train.csv', 'Titanic-ML-NB.ipynb', '.ipynb_checkpoints', 'test.csv', '.~lock.train.csv#']


In [2]:
training = pd.read_csv('train.csv')
testing = pd.read_csv('test.csv')

In [3]:
print(training.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [4]:
training.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
training.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
#Below columns don't offer characteristics to discern data
del training["Name"]
del testing["Name"]

del training["Ticket"]
del testing["Ticket"]

del training["PassengerId"]

In [7]:
#Make sure Null values are filled
training["Age"].fillna(-99, inplace=True)
testing["Age"].fillna(-99, inplace=True)

training["Embarked"].fillna(-99, inplace=True)
testing["Embarked"].fillna(-99, inplace=True)

training["Cabin"].fillna('Z', inplace=True)
testing["Cabin"].fillna('Z', inplace=True)

training["Fare"].fillna(training["Fare"].mean(), inplace=True)
testing["Fare"].fillna(training["Fare"].mean(), inplace=True)

In [8]:
#Replace Strings with number values
training = training.replace("male", 1)
training = training.replace("female", 0)
testing = testing.replace("male", 1)
testing = testing.replace("female", 0)

In [9]:
training = training.replace("S", 1)
training = training.replace("C", 2)
training = training.replace("Q", 3)

testing = testing.replace("S", 1)
testing = testing.replace("C", 2)
testing = testing.replace("Q", 3)

training["Embarked"] = training["Embarked"].astype(int)

In [10]:
#Below we will look at given features and see their relation to survival
training[["Survived","Sex"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,0,0.742038
1,1,0.188908


In [11]:
training["Sex"].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [12]:
training[["Pclass", "Survived"]].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [13]:
training[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [14]:
training[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [15]:
training[["Embarked", "Survived"]].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,-99,1.000000
2,2,0.553571
3,3,0.389610
1,1,0.336957


In [16]:
def labelAgeRange(row):
    "Labels row value with age value 1->8"
    #Unknown Age, Null filled with -99
    if row['Age'] == -99:
        return 1
    elif row['Age'] <= 10:
        return 2
    elif row['Age'] <= 15:
        return 3
    elif row['Age'] <= 20:
        return 4
    elif row['Age'] <= 25:
        return 5
    elif row['Age'] <= 30:
        return 6
    elif row['Age'] <= 35:
        return 7
    elif row['Age'] <= 40:
        return 8
    elif row['Age'] <= 45:
        return 9
    elif row['Age'] <= 50:
        return 10
    elif row['Age'] <= 55:
        return 11
    elif row['Age'] <= 60:
        return 12
    #Older than 60
    else:
        return 13

training["AgeRange"] = training.apply(labelAgeRange, axis=1)
testing["AgeRange"] = testing.apply(labelAgeRange, axis=1)

In [17]:
training[["AgeRange", "Survived"]].groupby(['AgeRange'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,AgeRange,Survived
1,2,0.593750
2,3,0.578947
6,7,0.465909
7,8,0.417910
10,11,0.416667
9,10,0.410256
5,6,0.388889
11,12,0.388889
8,9,0.361702
4,5,0.344262


In [18]:
def labelFareRange(row):
    "Labels row value with Fare range value 1->6"
    if row['Fare'] > 80.00:
        return 1
    elif row['Fare'] > 40.00:
        return 2
    elif row['Fare'] > 20.00:
        return 3
    elif row['Fare'] > 10.00:
        return 4
    elif row['Fare'] >= 7.125:
        return 5
    #Fare less than $7.125
    else:
        return 6

training["FareRange"] = training.apply(labelFareRange, axis=1)
testing["FareRange"] = testing.apply(labelFareRange, axis=1)

In [19]:
training[["FareRange", "Survived"]].groupby(['FareRange'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FareRange,Survived
0,1,0.770270
1,2,0.549020
2,3,0.430000
3,4,0.424581
4,5,0.218121
5,6,0.052632


In [20]:
def labelFamilySizeRange(row):
    "Labels row value with Family Size"
    familySize = row['SibSp'] + row['Parch'] +1
    
    if familySize == 1:
        return 1
    elif familySize <= 3:
        return 2
    elif familySize <= 5:
        return 3
    else:
        return 4

training["FamilySizeRange"] = training.apply(labelFamilySizeRange, axis=1)
testing["FamilySizeRange"] = testing.apply(labelFamilySizeRange, axis=1)

In [21]:
training[["FamilySizeRange", "Survived"]].groupby(['FamilySizeRange'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,FamilySizeRange,Survived
1,2,0.562738
2,3,0.545455
0,1,0.303538
3,4,0.148936


In [22]:
training[["Fare", "Survived"]].groupby(['Fare'], as_index=False).mean().sort_values(by='Survived', ascending=False).head()

,Fare,Survived
247,512.3292,1.0
196,57.9792,1.0
89,13.8583,1.0
88,13.7917,1.0
86,13.4167,1.0


In [23]:
def labelWealthFactor(row):
    "Labels row with corresponding wealth designation 1->6"
    if row['Fare'] >= 80.0 and row['Pclass'] == 1:
        return 1
    elif row['Pclass'] == 1:
        return 2
    elif row['Pclass'] == 2:
        return 3
    #3rd Class Below
    elif row['Fare'] >= 7.75:
        return 4
    elif row['Fare'] >= 7.125:
        return 5
    else:
        return 6

training["WealthFactor"] = training.apply(labelWealthFactor, axis=1)
testing["WealthFactor"] = testing.apply(labelWealthFactor, axis=1)

In [24]:
training[["WealthFactor", "Survived"]].groupby(['WealthFactor'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,WealthFactor,Survived
0,1,0.776316
1,2,0.550000
2,3,0.472826
3,4,0.256927
4,5,0.220588
5,6,0.076923


In [25]:
def labelIsAlone(row):
    "Labels row with 1 = Person is alone, 0 = is not alone"
    if row['FamilySizeRange'] == 1:
        return 1
    else:
        return 0

training["IsAlone"] = training.apply(labelIsAlone, axis=1)
testing["IsAlone"] = testing.apply(labelIsAlone, axis=1)

In [26]:
training[["IsAlone", "Survived"]].groupby(['IsAlone'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,IsAlone,Survived
0,0,0.505650
1,1,0.303538


In [27]:
def labelHasCabin(row):
    "0 = No cabin, 1 = has cabin"
    if row["Cabin"] == 'Z':
        return 0
    else:
        return 1

training["HasCabin"] = training.apply(labelHasCabin, axis=1)
testing["HasCabin"] = testing.apply(labelHasCabin, axis=1)

In [28]:
training[["HasCabin", "Survived"]].groupby(['HasCabin'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,HasCabin,Survived
1,1,0.666667
0,0,0.299854


In [29]:
def labelPriority(row):
    "Labels row with corresponding priority value 1->8"
    #Infants or Women in 1st Class with a Cabin
    if (row['Age'] <= 6 or row['Sex'] == 0) and (row['Pclass'] == 1 and row["HasCabin"] == 1):
        return 1
    #Infants or Women in 2nd Class
    elif (row['Age'] <= 6 or row['Sex'] == 0) and row["Pclass"] == 2:
        return 2
    #Infants or Women with Family Size between 2 and 3 who paid more tha $40 for ticket.
    elif (row['Age'] <= 6 or row['Sex'] == 0) and (row["FamilySizeRange"] == 2 and row["FareRange"] <= 2):
        return 3
    elif (row['Age'] <= 6 or row['Sex'] == 0) and row["FamilySizeRange"] == 2:
        return 4
    elif (row['Age'] <= 6 or row['Sex'] == 0) and row["Pclass"] == 1:
        return 5
    elif (row['Age'] <= 6 or row['Sex'] == 0) and row["Embarked"] == 2:
        return 6
    elif (row['Age'] <= 6 or row['Sex'] == 0):
        return 7
    #below represents men
    elif row["WealthFactor"] == 1 and row["IsAlone"] == 1 and row["Embarked"] == 2:
        return 8
    elif row["WealthFactor"] == 1 and row["IsAlone"] == 0 and row["Embarked"] == 2:
        return 9
    elif row["WealthFactor"] == 2 and row["IsAlone"] == 1 and row["Embarked"] == 2:
        return 10
    elif row["WealthFactor"] == 1 and row["IsAlone"] == 0 and row["Embarked"] == 1:
        return 11
    elif row["FamilySizeRange"] == 2:
        return 12
    else:
        return 13

training["Priority"] = training.apply(labelPriority, axis=1)
testing["Priority"] = testing.apply(labelPriority, axis=1)

In [30]:
training[["Priority", "Survived"]].groupby(['Priority'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Priority,Survived
2,3,1.000000
0,1,0.893617
1,2,0.860215
3,4,0.515152
4,5,0.500000
7,8,0.500000
10,11,0.500000
9,10,0.400000
5,6,0.375000
8,9,0.363636


In [31]:
training.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,AgeRange,FareRange,FamilySizeRange,WealthFactor,IsAlone,HasCabin,Priority
0,0,3,1,22.0,1,0,7.2500,Z,1,5,5,2,5,0,0,12
1,1,1,0,38.0,1,0,71.2833,C85,2,8,2,2,2,0,1,1
2,1,3,0,26.0,0,0,7.9250,Z,1,6,5,1,4,1,0,7
3,1,1,0,35.0,1,0,53.1000,C123,1,7,2,2,2,0,1,1
4,0,3,1,35.0,0,0,8.0500,Z,1,7,5,1,4,1,0,13


In [32]:
#Remove redundant columns which were used to create other feature columns
del training["Age"]
del training["SibSp"]
del training["Parch"]
del training["Fare"]
# del training["WealthFactor"]
# del training["Priority"]
# del training["IsAlone"]
del training["Cabin"]
# del training["FamilySizeRange"]
# del training["Embarked"]
del training["AgeRange"]
# del training["Sex"]
# del training["Pclass"]
del training["FareRange"]
# del training["HasCabin"]

del testing["Age"]
del testing["SibSp"]
del testing["Parch"]
del testing["Fare"]
# del testing["WealthFactor"]
# del testing["Priority"]
# del testing["IsAlone"]
del testing["Cabin"]
# del testing["FamilySizeRange"]
# del testing["Embarked"]
del testing["AgeRange"]
# del testing["Sex"]
# del testing["Pclass"]
del testing["FareRange"]
# del testing["HasCabin"]


In [33]:
# trainColumnNames = ['Survived','Priority','WealthFactor','Pclass','HasCabin','FareRange']
# testColumnNames = ['PassengerId','Priority','WealthFactor','Pclass','HasCabin','FareRange']
# training = training.reindex(columns=trainColumnNames)
# testing = testing.reindex(columns=testColumnNames)

In [34]:
training.head()

,Survived,Pclass,Sex,Embarked,FamilySizeRange,WealthFactor,IsAlone,HasCabin,Priority
0,0,3,1,1,2,5,0,0,12
1,1,1,0,2,2,2,0,1,1
2,1,3,0,1,1,4,1,0,7
3,1,1,0,1,2,2,0,1,1
4,0,3,1,1,1,4,1,0,13


In [35]:
testing.head()

,PassengerId,Pclass,Sex,Embarked,FamilySizeRange,WealthFactor,IsAlone,HasCabin,Priority
0,892,3,1,3,1,4,1,0,13
1,893,3,0,1,2,6,0,0,4
2,894,2,1,3,1,3,1,0,13
3,895,3,1,1,1,4,1,0,13
4,896,3,0,1,2,4,0,0,4


In [36]:
#Make sure training and testing are same schema
X_train = training.drop("Survived", axis=1)
Y_train = training["Survived"]
X_test  = testing.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 8), (891,), (418, 8))

In [37]:
#Feed our model to DecisionTreeClassifier from SciKit Learn
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

84.62

In [38]:
#Create submission dataframe
submission = pd.DataFrame({
        "PassengerId": testing["PassengerId"],
        "Survived": Y_pred
    })

In [39]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [40]:
#Save to file for uploading to Kaggle competition
submission.to_csv('submission.csv', index=False)